In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import math
import time
import random

## Read data file and some glimpse on raw data

In [2]:
filename = "experiment_data_lstm.csv"
df = pd.read_csv(filename, header=0, sep=',', dtype=np.float64)
print('Are ther any Nan value?:\n', np.isnan(df).any())
# ignore rows which contain zero
# ignore rows which have negative instantaneous rate
ignore_zero = df[(df['Strain']==0) | (df['Stress']==0) | (df['Temperature']==0) | (df['Strain Rate']<=0) ]
df=pd.concat([df, ignore_zero, ignore_zero]).drop_duplicates(keep=False)
df=pd.concat([df, ignore_zero, ignore_zero]).drop_duplicates(keep=False)
df.head()

Are ther any Nan value?:
 Strain               False
Stress               False
Strain Rate          False
Temperature          False
Normalized Stress    False
dtype: bool


,Strain,Stress,Strain Rate,Temperature,Normalized Stress
1,0.0002,580.282200,0.0001,298.0,1.000000
2,0.0004,644.631000,0.0001,298.0,1.110892
3,0.0006,681.693300,0.0001,298.0,1.174762
4,0.0008,706.112533,0.0001,298.0,1.216843
5,0.0010,727.073400,0.0001,298.0,1.252965


## Splitting flow curves

The individual flow curves are stacked one after the other in the csv file, so they need to be split according to the strain rate and temperature

In [3]:
# Flow curves in different strain rates at RT
df_e4 = df[(df['Strain Rate']==0.0001) & (df['Temperature']==298.0)]
df_e3 = df[(df['Strain Rate']==0.001) & (df['Temperature']==298.0)]
df_e2 = df[(df['Strain Rate']==0.01) & (df['Temperature']==298.0)]
df_e1 = df[(df['Strain Rate']==0.1) & (df['Temperature']==298.0)]

In [4]:
# Flow curves in 0.1 strain rate at varying T
df_e1_373K = df[(df['Strain Rate']==0.1) & (df['Temperature']==373.0)]
df_e1_473K = df[(df['Strain Rate']==0.1) & (df['Temperature']==473.0)]
df_e1_573K = df[(df['Strain Rate']==0.1) & (df['Temperature']==573.0)]
df_e1_673K = df[(df['Strain Rate']==0.1) & (df['Temperature']==673.0)]

In [5]:
# Flow curves in 0.01 strain rate at varying T
df_e2_373K = df[(df['Strain Rate']==0.01) & (df['Temperature']==373.0)]
df_e2_473K = df[(df['Strain Rate']==0.01) & (df['Temperature']==473.0)]
df_e2_573K = df[(df['Strain Rate']==0.01) & (df['Temperature']==573.0)]
df_e2_673K = df[(df['Strain Rate']==0.01) & (df['Temperature']==673.0)]

In [6]:
# Flow curves in 0.001 strain rate at varying T
df_e3_373K = df[(df['Strain Rate']==0.001) & (df['Temperature']==373.0)]
df_e3_473K = df[(df['Strain Rate']==0.001) & (df['Temperature']==473.0)]
df_e3_573K = df[(df['Strain Rate']==0.001) & (df['Temperature']==573.0)]
df_e3_673K = df[(df['Strain Rate']==0.001) & (df['Temperature']==673.0)]

In [7]:
# Flow curves in 0.0001 strain rate at varying T
df_e4_373K = df[(df['Strain Rate']==0.0001) & (df['Temperature']==373.0)]
df_e4_473K = df[(df['Strain Rate']==0.0001) & (df['Temperature']==473.0)]
df_e4_573K = df[(df['Strain Rate']==0.0001) & (df['Temperature']==573.0)]
df_e4_673K = df[(df['Strain Rate']==0.0001) & (df['Temperature']==673.0)]

In [8]:
# Put all flow curves in a dictionary for easy access 
flows = {'e4_298K': df_e4,
         'e4_373K': df_e4_373K,
         'e4_473K': df_e4_473K,
         'e4_573K': df_e4_573K,
         'e4_673K': df_e4_673K,
         'e3_298K': df_e3,
         'e3_373K': df_e3_373K,
         'e3_473K': df_e3_473K,
         'e3_573K': df_e3_573K,
         'e3_673K': df_e3_673K,
         'e2_298K': df_e2,
         'e2_373K': df_e2_373K,
         'e2_473K': df_e2_473K,
         'e2_573K': df_e2_573K,
         'e2_673K': df_e2_673K,
         'e1_298K': df_e1,
         'e1_373K': df_e1_373K,
         'e1_473K': df_e1_473K,
         'e1_573K': df_e1_573K,
         'e1_673K': df_e1_673K
        }
# Iterate through the flows dictionary and print the shape of each DataFrame
for key, df in flows.items():
    print(f"Shape of {key}: {df.shape}")

Shape of e4_298K: (241, 5)
Shape of e4_373K: (528, 5)
Shape of e4_473K: (567, 5)
Shape of e4_573K: (468, 5)
Shape of e4_673K: (512, 5)
Shape of e3_298K: (234, 5)
Shape of e3_373K: (526, 5)
Shape of e3_473K: (607, 5)
Shape of e3_573K: (772, 5)
Shape of e3_673K: (596, 5)
Shape of e2_298K: (94, 5)
Shape of e2_373K: (552, 5)
Shape of e2_473K: (533, 5)
Shape of e2_573K: (691, 5)
Shape of e2_673K: (579, 5)
Shape of e1_298K: (48, 5)
Shape of e1_373K: (61, 5)
Shape of e1_473K: (45, 5)
Shape of e1_573K: (64, 5)
Shape of e1_673K: (64, 5)


## Preprocessing

In this section, the data is first normalised, then split into train-test data, and finally split into features and labels.

In [9]:
# First, logarithm is applied to strain and strain rate before normalising.

flows_log = {}
for key, fc in flows.items():
    fc = fc.reset_index(drop=True)
    # apply decadic log to strain rates
    log_rate = pd.DataFrame(np.log10(fc['Strain Rate'].values))
    log_rate.columns = ['log_rate']
    # apply decadic log to strain
    log_strain = pd.DataFrame(np.log10(fc['Strain'].values))
    log_strain.columns = ['log_strain']
    
    fc_log = pd.concat([fc, log_strain, log_rate], axis=1)
    fc_log = fc_log.drop(columns=['Strain', 'Strain Rate'])
    cols = ['log_strain', 'log_rate', 'Temperature', 'Stress', 'Normalized Stress']
    fc_log = fc_log[cols]
    flows_log[key] = fc_log
raw_features = len(flows_log['e4_298K'].columns.values)
flows_log['e4_298K'].columns.values

array(['log_strain', 'log_rate', 'Temperature', 'Stress',
       'Normalized Stress'], dtype=object)

In [10]:
# Normalise the data according to the min-max formula

flows_scaled = {}
#stress_range = np.array([40, 570])
#temp_range = np.array([290, 700])
# Get the range for Stress
stress_min = df['Normalized Stress'].min()
stress_max = df['Normalized Stress'].max()
stress_range = np.array([stress_min, stress_max])

# Get the range for Temp (Temperature)
temp_min = df['Temperature'].min()
temp_max = df['Temperature'].max()
temp_range = np.array([298,673])

log_strain_range = [np.log10(0.08068), np.log10(1e-9)] # valid only if strain < 1.0
log_rate_range = [np.log10(0.1), np.log10(0.0001)] # valid only if rate < 1.0

def manual_scaling(feat, range_value):
    return (feat - range_value[0])/range_value[1]
def manual_descaling(feat, range_value): # inverse of the function above
    return feat * range_value[1] + range_value[0]

for key, fc in flows_log.items():
    fc['Normalized Stress'] = manual_scaling(fc['Normalized Stress'], stress_range)
    fc['Temperature'] = manual_scaling(fc['Temperature'], temp_range)
    fc['log_strain'] = manual_scaling(fc['log_strain'], log_strain_range)
    fc['log_rate'] = manual_scaling(fc['log_rate'], log_rate_range)
    flows_scaled[key] = fc
flows_scaled['e4_373K'].head()

,log_strain,log_rate,Temperature,Stress,Normalized Stress
0,0.400637,0.75,0.111441,571.162231,0.036753
1,0.434085,0.75,0.111441,579.519959,0.045409
2,0.367190,0.75,0.111441,587.720642,0.053902
3,0.356422,0.75,0.111441,595.775391,0.062244
4,0.381072,0.75,0.111441,603.783569,0.070537


## Decompose train and test set

In [11]:
'''
train_set_name = ['e4_298K', 'e4_373K', 'e4_573K', 'e4_673K', 
                  'e3_373K', 'e3_473K', 'e3_573K', 'e3_673K',
                  'e2_298K', 'e2_473K','e2_573K', 'e2_673K', 
                  'e1_298K', 'e1_373K', 'e1_473K', 'e1_673K']
test_set_name = ['e4_473K',
                 'e3_298K', 
                 'e2_373K', 
                 'e1_573K']
'''
import random

# Group keys by their prefixes (e4, e3, e2, e1)
groups = {
    'e4': [key for key in flows_scaled.keys() if key.startswith('e4')],
    'e3': [key for key in flows_scaled.keys() if key.startswith('e3')],
    'e2': [key for key in flows_scaled.keys() if key.startswith('e2')],
    'e1': [key for key in flows_scaled.keys() if key.startswith('e1')]
}

# Ensure test set includes one key per group with unique temperatures
selected_temperatures = set()
test_set_name = []

for group in groups:
    group_keys = groups[group]
    valid_keys = [key for key in group_keys if key.split('_')[-1] not in selected_temperatures]
    
    if valid_keys:  # If there are keys with unselected temperatures
        chosen_key = random.choice(valid_keys)
        selected_temperatures.add(chosen_key.split('_')[-1])
        test_set_name.append(chosen_key)
    else:
        # Fallback to random choice if no unique temperature is left
        test_set_name.append(random.choice(group_keys))

# The remaining keys will be used for the training set
train_set_name = [key for key in flows_scaled.keys() if key not in test_set_name]

# Output the results
print("Test Set:", test_set_name)
print("Train Set:", train_set_name)

train_set = []
test_set = []
val_set = []
exp_no_T = []

for key, fc in flows_scaled.items():
    if key in train_set_name:
        train_set.append(fc)
    if key in test_set_name:
        test_set.append(fc)
    else:
        exp_no_T.append(fc)
        
df_train = pd.concat(train_set)
df_test = pd.concat(test_set)

Test Set: ['e4_473K', 'e3_373K', 'e2_298K', 'e1_573K']
Train Set: ['e4_298K', 'e4_373K', 'e4_573K', 'e4_673K', 'e3_298K', 'e3_473K', 'e3_573K', 'e3_673K', 'e2_373K', 'e2_473K', 'e2_573K', 'e2_673K', 'e1_298K', 'e1_373K', 'e1_473K', 'e1_673K']


In [12]:
flows_scaled.keys()

dict_keys(['e4_298K', 'e4_373K', 'e4_473K', 'e4_573K', 'e4_673K', 'e3_298K', 'e3_373K', 'e3_473K', 'e3_573K', 'e3_673K', 'e2_298K', 'e2_373K', 'e2_473K', 'e2_573K', 'e2_673K', 'e1_298K', 'e1_373K', 'e1_473K', 'e1_573K', 'e1_673K'])

## Decompose label and feature

In [13]:
lbl_name = ['Normalized Stress']

ignore_cols = ['Stress']

def decomp_label_feat(df, lbl_col, ignore_cols=[]):
    label = pd.DataFrame(df[lbl_col])
    ignore_cols.append(lbl_col[0])
    feat = df.drop(columns=ignore_cols)
    return feat.values, label.values

def restore_strain(log_strain_col, range_value):
    log_strain = manual_descaling(log_strain_col, log_strain_range)
    return 10**log_strain

print('Original columns: \n\n',cols,'\n')
print('Label column...', lbl_name, '\n')
print('Ignore columns...', ignore_cols, '\n')


X_train, y_train = decomp_label_feat(df_train, lbl_name, ignore_cols)
print('X_train shape:', X_train.shape)
print('y_train shape:', y_train.shape)
X_test, y_test = decomp_label_feat(df_test, lbl_name, ignore_cols)
print('X_test shape:', X_test.shape)
print('y_test shape:', y_test.shape)


# Organise the training and testing flow curves into their respective dictionaries for easy access
X_test_dict = {}
y_test_dict = {}
for ii, df_test in enumerate(test_set):
    X_test_dict[test_set_name[ii]], y_test_dict[test_set_name[ii]] = decomp_label_feat(df_test, lbl_name, ignore_cols)
    print('X_test shape for set ' + test_set_name[ii], X_test_dict[test_set_name[ii]].shape)
    print('y_test shape for set ' + test_set_name[ii], y_test_dict[test_set_name[ii]].shape)

X_train_dict = {}
y_train_dict = {}
for jj, df_train in enumerate(train_set):
    X_train_dict[train_set_name[jj]], y_train_dict[train_set_name[jj]] = decomp_label_feat(df_train, lbl_name, ignore_cols)
    print('X_train shape for set ' + train_set_name[jj], X_train_dict[train_set_name[jj]].shape)
    print('y_train shape for set ' + train_set_name[jj], y_train_dict[train_set_name[jj]].shape)

Original columns: 

 ['log_strain', 'log_rate', 'Temperature', 'Stress', 'Normalized Stress'] 

Label column... ['Normalized Stress'] 

Ignore columns... ['Stress'] 

X_train shape: (6531, 3)
y_train shape: (6531, 1)
X_test shape: (1251, 3)
y_test shape: (1251, 1)
X_test shape for set e4_473K (567, 3)
y_test shape for set e4_473K (567, 1)
X_test shape for set e3_373K (526, 3)
y_test shape for set e3_373K (526, 1)
X_test shape for set e2_298K (94, 3)
y_test shape for set e2_298K (94, 1)
X_test shape for set e1_573K (64, 3)
y_test shape for set e1_573K (64, 1)
X_train shape for set e4_298K (241, 3)
y_train shape for set e4_298K (241, 1)
X_train shape for set e4_373K (528, 3)
y_train shape for set e4_373K (528, 1)
X_train shape for set e4_573K (468, 3)
y_train shape for set e4_573K (468, 1)
X_train shape for set e4_673K (512, 3)
y_train shape for set e4_673K (512, 1)
X_train shape for set e3_298K (234, 3)
y_train shape for set e3_298K (234, 1)
X_train shape for set e3_473K (607, 3)
y_trai

## Stacked LSTM

In [14]:
pip install scikeras

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [15]:
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import r2_score

2024-12-19 13:14:43.957586: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-19 13:14:43.976692: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-19 13:14:43.999866: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-19 13:14:44.006986: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-19 13:14:44.025965: I tensorflow/core/platform/cpu_feature_guar

In [16]:
# utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [17]:
def sLSTMmodel(input_shape, nNeurons_1=150, nNeurons_2=370, nNeurons_3=370, out_dim=1):
    model = Sequential()
    model.add(LSTM(nNeurons_1, input_shape=input_shape, activation='relu', return_sequences=True))
    #model.add(Dropout(0.5))
    model.add(LSTM(nNeurons_2, input_shape=input_shape, activation='relu', return_sequences=True))
    #model.add(Dropout(0.5))
    model.add(LSTM(nNeurons_3, input_shape=input_shape, activation='relu', return_sequences=True))
    #model.add(Dropout(0.5))
    model.add(Dense(out_dim))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [18]:
# Hyperparameter tuning 
# Keep runLSTM as True if you want to perform the hyperparameter tuning
runLSTM = False
np.random.seed(157)
n_iter = 10
param_dist = {
    'model__nNeurons_1': range(10, 300, 20),
    'model__nNeurons_2': range(10, 500, 20),
    'model__nNeurons_3': range(10, 500, 20)
}

X_train_lstm = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
if runLSTM:
    slstm = KerasRegressor(model=sLSTMmodel, input_shape=X_train_lstm.shape[1:], epochs=50, batch_size=32, verbose=0)
    random_search_slstm = RandomizedSearchCV(estimator=slstm, param_distributions=param_dist, scoring='r2')
    start = time.time()
    random_search_slstm.fit(X_train_lstm, y_train)
    elapsed_lstm = (time.time() - start)
    print("RandomizedSearchCV took %.2f seconds for %d candidates"
          " parameter settings." % (elapsed_lstm, n_iter))
    report(random_search_slstm.cv_results_)

# This will output the 3 best models along with the time taken to complete the hyperparameter tuning.
# It is best to run this using GPUs on CSC.

In [19]:
# training
runLSTM = False
X_train_lstm = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test_lstm = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))
if runLSTM:
    best_slstm = random_search_slstm.best_estimator_
else:
    best_slstm_param =  {'nNeurons_3': 370, 'nNeurons_2': 370, 'nNeurons_1': 150}
    best_slstm = KerasRegressor(model=sLSTMmodel, input_shape=X_train_lstm.shape[1:], epochs=50, batch_size=32,  verbose=0)
start = time.time()
best_slstm.fit(X_train_lstm, y_train)
elapsed_slstm_test = (time.time() - start)
print('r2_score for training set:', r2_score(y_train, best_slstm.predict(X_train_lstm)))
print('r2_score for testing set:', r2_score(y_test, best_slstm.predict(X_test_lstm)))
print("Stack LSTM took %.2f seconds for test." % (elapsed_slstm_test))
slstm_test_pred = {}
for key, X in X_test_dict.items():
    X = np.reshape(X, (X.shape[0], 1, X.shape[1]))
    slstm_test_pred[key] = manual_descaling(best_slstm.predict(X), stress_range)

2024-12-19 13:14:46.513804: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31141 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:8a:00.0, compute capability: 7.0
/users/lehoang1/.local/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
I0000 00:00:1734606891.790277 3389785 service.cc:146] XLA service 0x561a38864050 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1734606891.790305 3389785 service.cc:154]   StreamExecutor device (0): Tesla V100-SXM2-32GB, Compute Capability 7.0
2024-12-19 13:14:51.890452: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_

r2_score for training set: 0.993913022232035
r2_score for testing set: 0.9831200659950813
Stack LSTM took 24.51 seconds for test.


In [20]:
# Run this code to skip hyperparameter tuning and training. my_model_final.h5 contains the trained model. 
#best_slstm.model_.save('lstm_model.h5')
#from keras.models import load_model
#best_slstm = load_model('lstm_model.h5')

## Results

### Training results

In [ ]:
# Make the predictions for each training flow curve
slstm_train_pred_stress = {}

for key, X in X_train_dict.items():
    X = np.reshape(X, (X.shape[0], 1, X.shape[1]))
    slstm_train_pred_stress[key] = manual_descaling(best_slstm.predict(X)[:,0], stress_range)

#### Stress predictions

In [ ]:
plt.figure(figsize=(6,6))

plt.plot(flows['e4_298K']['Strain'][::15], flows['e4_298K']['Normalized Stress'][::15], 'o', label='298 K', color='red', alpha=1.0)
plt.plot(flows['e4_298K']['Strain'], slstm_train_pred_stress['e4_298K'][:,], label='298 K Prediction', color='red', alpha=1.0, linewidth=2.5)

plt.plot(flows['e4_373K']['Strain'][::20], flows['e4_373K']['Normalized Stress'][::20], 'o', label='373 K', color='blue', alpha=1.0)
plt.plot(flows['e4_373K']['Strain'], slstm_train_pred_stress['e4_373K'][:,], label='373 K Prediction', color='blue', alpha=1.0, linewidth=2.5)

plt.plot(flows['e4_573K']['Strain'][::15], flows['e4_573K']['Normalized Stress'][::15], 'o', label='573 K', color='green', alpha=1.0)
plt.plot(flows['e4_573K']['Strain'], slstm_train_pred_stress['e4_573K'][:,], label='573 K Prediction', color='green', alpha=1.0, linewidth=2.5)

plt.plot(flows['e4_673K']['Strain'][::20], flows['e4_673K']['Normalized Stress'][::20], 'o', label='673 K', color='orange', alpha=1.0)
plt.plot(flows['e4_673K']['Strain'], slstm_train_pred_stress['e4_673K'][:,], label='673 K Prediction', color='orange', alpha=1.0, linewidth=2.5)

plt.gca().add_artist(plt.legend(loc='best', fontsize='14'))
plt.xlabel('True strain, -', fontsize='14')    #Name x label
plt.ylabel('Normalized stress, -', fontsize='14')    #Name y label

plt.xticks(fontsize='14')
plt.yticks(fontsize='14')
plt.xlim(xmin = 0)
plt.ylim(0.6, 2)
plt.title('Strain rates 0.0001 /s', fontweight='bold', fontsize='14')
plt.savefig('FC_Train_0.0001.png')

In [ ]:
plt.figure(figsize=(6,6))

#plt.plot(flows['e3']['Strain'][::20], flows['e3']['Normalized Stress'][::20], '.', label='298 K', color='red', alpha=1.0, markersize=10)
#plt.plot(flows['e3']['Strain'], slstm_train_pred_stress['e3'][:,], label='298 K Prediction', color='red', alpha=1.0, linewidth=2.5)

plt.plot(flows['e3_373K']['Strain'][::20], flows['e3_373K']['Normalized Stress'][::20], 'o', label='373 K', color='red', alpha=1.0)
plt.plot(flows['e3_373K']['Strain'], slstm_train_pred_stress['e3_373K'][:,], label='373 K Prediction', color='red', alpha=1.0, linewidth=2.5)

plt.plot(flows['e3_473K']['Strain'][::20], flows['e3_473K']['Normalized Stress'][::20], 'o', label='473 K', color='blue', alpha=1.0)
plt.plot(flows['e3_473K']['Strain'], slstm_train_pred_stress['e3_473K'][:,], label='473 K Prediction', color='blue', alpha=1.0, linewidth=2.5)

plt.plot(flows['e3_573K']['Strain'][::20], flows['e3_573K']['Normalized Stress'][::20], 'o', label='573 K', color='green', alpha=1.0)
plt.plot(flows['e3_573K']['Strain'], slstm_train_pred_stress['e3_573K'][:,], label='573 K Prediction', color='green', alpha=1.0, linewidth=2.5)

plt.plot(flows['e3_673K']['Strain'][::20], flows['e3_673K']['Normalized Stress'][::20], 'o', label='673 K', color='orange', alpha=1.0)
plt.plot(flows['e3_673K']['Strain'], slstm_train_pred_stress['e3_673K'][:,], label='673 K Prediction', color='orange', alpha=1.0, linewidth=2.5)

plt.gca().add_artist(plt.legend(loc='best', fontsize='14'))
plt.xlabel('True strain, -', fontsize='14')    #Name x label
plt.ylabel('Normalized stress, -', fontsize='14')    #Name y label

plt.xticks(fontsize='14')
plt.yticks(fontsize='14')
plt.xlim(xmin = 0)
plt.ylim(0.6, 2)
plt.title('Strain rates 0.001 /s', fontweight='bold', fontsize='14')
plt.savefig('FC_Train_0.001.png')

In [ ]:
plt.figure(figsize=(6,6))

plt.plot(flows['e2_298K']['Strain'][::4], flows['e2_298K']['Normalized Stress'][::4], 'o', label='298 K', color='red', alpha=1.0)
plt.plot(flows['e2_298K']['Strain'], slstm_train_pred_stress['e2_298K'][:,], label='298 K Prediction', color='red', alpha=1.0, linewidth=2.5)

#plt.plot(flows['e2_373K']['Strain'][::20], flows['e2_373K']['Normalized Stress'][::20], '.', label='373 K', color='blue', alpha=1.0, markersize=10)
#plt.plot(flows['e2_373K']['Strain'], slstm_train_pred_stress['e2_373K'][:,], label='373 K Prediction', color='blue', alpha=1.0, linewidth=2.5)

plt.plot(flows['e2_473K']['Strain'][::20], flows['e2_473K']['Normalized Stress'][::20], 'o', label='473 K', color='blue', alpha=1.0)
plt.plot(flows['e2_473K']['Strain'], slstm_train_pred_stress['e2_473K'][:,], label='473 K Prediction', color='blue', alpha=1.0, linewidth=2.5)

plt.plot(flows['e2_573K']['Strain'][::20], flows['e2_573K']['Normalized Stress'][::20], 'o', label='573 K', color='green', alpha=1.0)
plt.plot(flows['e2_573K']['Strain'], slstm_train_pred_stress['e2_573K'][:,], label='573 K Prediction', color='green', alpha=1.0, linewidth=2.5)

plt.plot(flows['e2_673K']['Strain'][::20], flows['e2_673K']['Normalized Stress'][::20], 'o', label='673 K', color='orange', alpha=1.0)
plt.plot(flows['e2_673K']['Strain'], slstm_train_pred_stress['e2_673K'][:,], label='673 K Prediction', color='orange', alpha=1.0, linewidth=2.5)

plt.gca().add_artist(plt.legend(loc='best', fontsize='14'))
plt.xlabel('True strain, -', fontsize='14')    #Name x label
plt.ylabel('Normalized stress, -', fontsize='14')    #Name y label

plt.xticks(fontsize='14')
plt.yticks(fontsize='14')
plt.xlim(xmin = 0)
plt.ylim(0.6, 2)
plt.title('Strain rates 0.01 /s', fontweight='bold', fontsize='14')
plt.savefig('FC_Train_0.01.png')

In [ ]:
plt.figure(figsize=(6,6))

plt.plot(flows['e1_298K']['Strain'][::2], flows['e1_298K']['Normalized Stress'][::2], 'o', label='298 K', color='red', alpha=1.0)
plt.plot(flows['e1_298K']['Strain'], slstm_train_pred_stress['e1_298K'], label='298 K Prediction', color='red', alpha=1.0, linewidth=2.5)

plt.plot(flows['e1_373K']['Strain'][::2], flows['e1_373K']['Normalized Stress'][::2], 'o', label='373 K', color='blue', alpha=1.0)
plt.plot(flows['e1_373K']['Strain'], slstm_train_pred_stress['e1_373K'], label='373 K Prediction', color='blue', alpha=1.0, linewidth=2.5)

plt.plot(flows['e1_473K']['Strain'][::2], flows['e1_473K']['Normalized Stress'][::2], 'o', label='473 K', color='green', alpha=1.0)
plt.plot(flows['e1_473K']['Strain'], slstm_train_pred_stress['e1_473K'], label='473 K Prediction', color='green', alpha=1.0, linewidth=2.5)

#plt.plot(flows['e1_573K']['Strain'][::2], flows['e1_573K']['Normalized Stress'][::2], '.', label='573 K', color='green', alpha=1.0, markersize=10)
#plt.plot(flows['e1_573K']['Strain'], slstm_train_pred_stress['e1_573K'], label='573 K Prediction', color='green', alpha=1.0, linewidth=2.5)

plt.plot(flows['e1_673K']['Strain'][::2], flows['e1_673K']['Normalized Stress'][::2], 'o', label='673 K', color='orange', alpha=1.0)
plt.plot(flows['e1_673K']['Strain'], slstm_train_pred_stress['e1_673K'][:,], label='673 K Prediction', color='orange', alpha=1.0, linewidth=2.5)

plt.gca().add_artist(plt.legend(loc='best', fontsize='14'))
plt.xlabel('True strain, -', fontsize='14')    #Name x label
plt.ylabel('Normalized stress, -', fontsize='14')    #Name y label

plt.xticks(fontsize='14')
plt.yticks(fontsize='14')
plt.xlim(xmin = 0)
plt.ylim(0.6, 2)
plt.title('Strain rates 0.1 /s', fontweight='bold', fontsize='14')
plt.savefig('FC_Train_0.1.png')

### Testing results

In [ ]:
slstm_test_pred_stress = {}
#slstm_test_pred_temp = {}
for key, X in X_test_dict.items():
    X = np.reshape(X, (X.shape[0], 1, X.shape[1]))
    slstm_test_pred_stress[key] = manual_descaling(best_slstm.predict(X)[:,0], stress_range)
    #slstm_test_pred_temp[key] = manual_descaling(best_slstm.predict(X)[:,1], delT_range)

#### Stress predictions

In [ ]:
plt.figure(figsize=(6,6))

plt.plot(flows['e4_473K']['Strain'][::20], flows['e4_473K']['Normalized Stress'][::20], 'o', label='473 K', color='red', alpha=1.0)
plt.plot(flows['e4_473K']['Strain'], slstm_test_pred_stress['e4_473K'][:,], label='473 K Prediction', color='red', alpha=1.0, linewidth=2.5)

plt.gca().add_artist(plt.legend(loc='best', fontsize='14'))
plt.xlabel('True strain, -', fontsize='14')    #Name x label
plt.ylabel('Normalized stress, -', fontsize='14')    #Name y label

plt.xticks(fontsize='14')
plt.yticks(fontsize='14')
plt.xlim(xmin = 0)
plt.ylim(0.6, 2)
plt.title('Strain rates 0.0001 /s', fontweight='bold',fontsize='14')
plt.savefig('FC_Train_0.0001.png')

In [ ]:
plt.figure(figsize=(6,6))

plt.plot(flows['e3_298K']['Strain'][::10], flows['e3_298K']['Normalized Stress'][::10], 'o', label='298 K', color='red', alpha=1.0)
plt.plot(flows['e3_298K']['Strain'], slstm_test_pred_stress['e3_298K'][:,], label='298 K Prediction', color='red', alpha=1.0, linewidth=2.5)

#plt.plot(flows['e3_473K']['Strain'][::20], flows['e3_473K']['Stress'][::20], '.', label='473 K', color='green', alpha=1.0, markersize=10)
#plt.plot(flows['e3_473K']['Strain'], slstm_test_pred_stress['e3_473K'][:,], label='473 K Prediction', color='green', alpha=1.0, linewidth=2.5)

#plt.plot(flows['e3_673K']['Strain'][::20], flows['e3_673K']['Stress'][::20], '.', label='673 K', color='blue', alpha=1.0, markersize=10)
#plt.plot(flows['e3_673K']['Strain'], slstm_test_pred_stress['e3_673K'][:,], label='673 K Prediction', color='blue', alpha=1.0, linewidth=2.5)

plt.gca().add_artist(plt.legend(loc='best', fontsize='14'))
plt.xlabel('True strain, -', fontsize='14')    #Name x label
plt.ylabel('Normalized stress, -', fontsize='14')    #Name y label

plt.xticks(fontsize='14')
plt.yticks(fontsize='14')
plt.xlim(xmin = 0)
plt.ylim(0.6, 2)
plt.title('Strain rates 0.001 /s', fontweight='bold', fontsize='14')
plt.savefig('FC_Train_0.001.png')

In [ ]:
plt.figure(figsize=(6,6))

plt.plot(flows['e2_373K']['Strain'][::20], flows['e2_373K']['Normalized Stress'][::20], 'o', label='373 K', color='red', alpha=1.0)
plt.plot(flows['e2_373K']['Strain'], slstm_test_pred_stress['e2_373K'][:,], label='373 K Prediction', color='red', alpha=1.0, linewidth=2.5)

#plt.plot(flows['e2_573K']['Strain'][::20], flows['e2_573K']['Stress'][::20], '.', label='573 K', color='green', alpha=1.0, markersize=10)
#plt.plot(flows['e2_573K']['Strain'], slstm_test_pred_stress['e2_573K'][:,], label='573 K Prediction', color='green', alpha=1.0, linewidth=2.5)

#plt.plot(flows['e2_673K']['Strain'][::20], flows['e2_673K']['Normalized Stress'][::20], '.', label='673 K', color='blue', alpha=1.0, markersize=10)
#plt.plot(flows['e2_673K']['Strain'], slstm_test_pred_stress['e2_673K'][:,], label='673 K Prediction', color='blue', alpha=1.0, linewidth=2.5)

plt.gca().add_artist(plt.legend(loc='best', fontsize='14'))
plt.xlabel('True strain, -', fontsize='14')    #Name x label
plt.ylabel('Normalized stress, -', fontsize='14')    #Name y label

plt.xticks(fontsize='14')
plt.yticks(fontsize='14')
plt.xlim(xmin = 0)
plt.ylim(0.6, 2)
plt.title('Strain rates 0.01 /s', fontweight='bold', fontsize='14')
plt.savefig('FC_Train_0.01.png')

In [ ]:
plt.figure(figsize=(6,6))

#plt.plot(flows['e1']['Strain'][::5], flows['e1']['Stress'][::5], '.', label='298 K', color='red', alpha=1.0, markersize=10)
#plt.plot(flows['e1']['Strain'], slstm_test_pred_stress['e1'][:,], label='298 K Prediction', color='red', alpha=1.0, linewidth=2.5)

#plt.plot(flows['e1_373K']['Strain'][::5], flows['e1_373K']['Stress'][::5], '.', label='373 K', color='blue', alpha=1.0, markersize=10)
#plt.plot(flows['e1_373K']['Strain'], slstm_test_pred_stress['e1_373K'][:,], label='373 K Prediction', color='blue', alpha=1.0, linewidth=2.5)

#plt.plot(flows['e1_473K']['Strain'][::2], flows['e1_473K']['Stress'][::2], '.', label='473 K', color='green', alpha=1.0, markersize=10)
#plt.plot(flows['e1_473K']['Strain'], slstm_test_pred_stress['e1_473K'][:,], label='473 K Prediction', color='green', alpha=1.0, linewidth=2.5)

plt.plot(flows['e1_573K']['Strain'][::2], flows['e1_573K']['Normalized Stress'][::2], 'o', label='573 K', color='red', alpha=1.0)
plt.plot(flows['e1_573K']['Strain'], slstm_test_pred_stress['e1_573K'][:,], label='573 K Prediction', color='red', alpha=1.0, linewidth=2.5)

#plt.plot(flows['e1_673K']['Strain'][::2], flows['e1_673K']['Stress'][::2], '.', label='673 K', color='blue', alpha=1.0, markersize=10)
#plt.plot(flows['e1_673K']['Strain'], slstm_test_pred_stress['e1_673K'][:,], label='673 K Prediction', color='blue', alpha=1.0, linewidth=2.5)

plt.gca().add_artist(plt.legend(loc='best', fontsize='14'))
plt.xlabel('True strain, -', fontsize='14')    #Name x label
plt.ylabel('Normalized stress, -', fontsize='14')    #Name y label

plt.xticks(fontsize='14')
plt.yticks(fontsize='14')
plt.xlim(xmin = 0)
plt.ylim(0.6, 2)
plt.title('Strain rates 0.1 /s', fontweight='bold', fontsize='14')
plt.savefig('FC_Train_0.1.png')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from mpl_toolkits.mplot3d import Axes3D

# Load the data
file_path = 'experiment_data_lstm.csv'  # Replace with your file path
data = pd.read_csv(file_path)

# Get unique strain rates
strain_rates = data['Strain Rate'].unique()

# Loop through each strain rate and create a smooth 3D plot
for strain_rate in strain_rates:
    # Filter the data for the current strain rate
    filtered_data = data[data['Strain Rate'] == strain_rate]

    # Prepare the data
    strain = filtered_data['Strain'].values
    temperature = filtered_data['Temperature'].values
    normalized_stress = filtered_data['Stress'].values

    # Create a higher-resolution grid for smooth interpolation
    strain_linspace = np.linspace(min(strain), max(strain), 100)
    temp_linspace = np.linspace(min(temperature), max(temperature), 100)
    strain_grid, temp_grid = np.meshgrid(strain_linspace, temp_linspace)

    # Interpolate the normalized stress data to the high-res grid
    normalized_stress_grid = griddata(
        points=(strain, temperature),
        values=normalized_stress,
        xi=(strain_grid, temp_grid),
        method='cubic'  # Smoother interpolation
    )

    # Plot the 3D surface
    fig = plt.figure(figsize=(10, 7))
    ax = fig.add_subplot(111, projection='3d')

    # Create the surface plot
    surf = ax.plot_surface(temp_grid, strain_grid, normalized_stress_grid, cmap='viridis', edgecolor='none')

    # Add labels and title
    ax.set_xlabel('Temperature (K)', fontsize=12)
    ax.set_ylabel('Strain (-)', fontsize=12)
    ax.set_zlabel('Stress (-)', fontsize=12)
    ax.set_title(f'Normalized Stress vs. Strain and Temperature\nStrain Rate: {strain_rate} /s', fontsize=14)

    # Add a color bar
    fig.colorbar(surf, shrink=0.5, aspect=10)

    # Show the plot
    plt.show()
